In [37]:
from sqlalchemy import create_engine, inspect, Column, Integer, String, ForeignKey, Boolean, DECIMAL, CheckConstraint
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker 
from pprint import pprint
from bs4 import BeautifulSoup
import requests
import pandas as pd
import json

In [46]:
Base = declarative_base()

class Estado(Base):
    __tablename__ = 'Estado'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Coluna obrigatória para chave primária
    estado = Column(String(45), unique=True)  # Define a length of 100 for the 'estado' column
    sigla = Column(String(3),  unique=True)    # Define a length of 2 for the 'sigla' column
    regiao = Column(String(50))  # Define a length of 50 for the 'regiao' column
    pais = Column(String(50))    # Define a length of 50 for the 'pais' column

class Cliente(Base):
    __tablename__ = 'Cliente'  # Nome da tabela no banco de dados
    id = Column(Integer, primary_key=True, autoincrement=True)  # Coluna obrigatória para chave primária
    estado_id = Column(Integer, ForeignKey('Estado.id'))  # Chave estrangeira referenciando a tabela estado
    sexo = Column(String(15))  # Define a length of 1 para o sexo
    idade = Column(Integer)  # Define a column for idade
    qtd_filhos = Column(Integer)  # Define a column for quantidade de filhos
    hipertenso = Column(Boolean)  # Define o tipo booleano para hipertenso
    diabetico = Column(Boolean)  # Define o tipo booleano para diabetico

    __table_args__ = (
        CheckConstraint('idade >= 0', name='check_idade_nao_negativo'),
        CheckConstraint('qtd_filhos >= 0', name='check_qtd_filhos_nao_negativo'),
    )


class ClasseProduto(Base):
    __tablename__ = 'ClasseProduto'  # Nome da tabela no banco de dados
    id = Column(Integer, primary_key=True, autoincrement=True)
    classe = Column(String(50), unique=True, nullable=False)  # Define a length of 50 para a classe do produto

class Produto(Base):
    __tablename__ = 'Produto'  # Nome da tabela no banco de dados
    id = Column(Integer, primary_key=True, autoincrement=True)  # Removed autoincrement=True
    classe_produto_id = Column(Integer, ForeignKey('ClasseProduto.id'))  # Chave estrangeira referenciando a tabela classe_produto
    produto = Column(String(50), unique=True, nullable=False)  # Define a length of 50 para o produto

class Compra(Base):
    __tablename__ = 'Compra'  # Nome da tabela no banco de dados
    id = Column(Integer, primary_key=True, autoincrement=True)  # Coluna obrigatória para chave primária
    cliente_id = Column(Integer, ForeignKey('Cliente.id'))  # Chave estrangeira referenciando a tabela cliente
    produto_id = Column(Integer, ForeignKey('Produto.id'))  # Chave estrangeira referenciando a tabela produto
    valor_unitario = Column(DECIMAL(10, 2))  # Define o tipo decimal com 2 casas decimais
    valor_total = Column(DECIMAL(10,2))  # Define o tipo inteiro para o valor da compra

    __table_args__ = (
        CheckConstraint('valor_unitario >= 0', name='check_valor_unitario_nao_negativo'),
        CheckConstraint('valor_total >= 0', name='check_valor_total_nao_negativo'),
    )

engine = create_engine('sqlite:///estudo.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

inspector = inspect(engine)
print(inspector.get_table_names())

['ClasseProduto', 'Cliente', 'Compra', 'Estado', 'Produto', 'classes_produto', 'clientes', 'compras', 'estados']


In [39]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

url ='https://leandrolessa.com.br/coleta-de-dados-na-web/'

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')

# Localiza a tabela pelo ID
table = soup.find('table', {'id': 'tablepress-1'})

# Extrai os cabeçalhos da tabela
headers = [header.text.strip() for header in table.find_all('th')]

# Extrai as linhas da tabela
rows = table.find_all('tr')[1:]  # Ignora o cabeçalho
print(rows)
# Preenche diretamente em uma variável tipo lista de dicionário
data = []
for row in rows:
    values = [cell.text.strip() for cell in row.find_all('td')]
    data.append(dict(zip(headers, values)))

# Converte para JSON e depois para lista de dicionários
json_data = json.dumps(data)
dict_list = json.loads(json_data)

# Exibe o resultado
pprint(dict_list)

[<tr class="row-2">
<td class="column-1"><span style="color:#303030;">1000</span></td><td class="column-2"><span style="color:#303030;">Sabão em pó</span></td><td class="column-3"><span style="color:#303030;">1</span></td><td class="column-4"><span style="color:#303030;">Produtos de Limpeza</span></td><td class="column-5"><span style="color:#303030;">9.9</span></td>
</tr>, <tr class="row-3">
<td class="column-1"><span style="color:#303030;">1001</span></td><td class="column-2"><span style="color:#303030;">Amaciante</span></td><td class="column-3"><span style="color:#303030;">1</span></td><td class="column-4"><span style="color:#303030;">Produtos de Limpeza</span></td><td class="column-5"><span style="color:#303030;">22</span></td>
</tr>, <tr class="row-4">
<td class="column-1"><span style="color:#303030;">1002</span></td><td class="column-2"><span style="color:#303030;">Detergente</span></td><td class="column-3"><span style="color:#303030;">1</span></td><td class="column-4"><span style

In [32]:
print("Total de registros:", len(dict_list))
valores_nulos = [dict_produto for dict_produto in dict_list if dict_produto['cod_produto'] == '']
print("Total de valores nulos:", len(valores_nulos))
print("Total de valores nulos:", len(valores_nulos))

Total de registros: 87
Total de valores nulos: 0
Total de valores nulos: 0


In [40]:
# Criação do DataFrame a partir dos valores extraídos
df = pd.DataFrame(dict_list)

# Exibe as primeiras linhas do DataFrame
print(df.head())

  cod_produto    nome_produto cod_classe_produto       classe_produto valor
0        1000     Sabão em pó                  1  Produtos de Limpeza   9.9
1        1001       Amaciante                  1  Produtos de Limpeza    22
2        1002      Detergente                  1  Produtos de Limpeza  1.99
3        1003  Água sanitária                  1  Produtos de Limpeza    12
4        1004  Esponja de aço                  1  Produtos de Limpeza     4


In [47]:
df_filtro_nulos = df.dropna()
print(len(df_filtro_nulos)) 

87


In [48]:
# Carrega o arquivo CSV em um DataFrame
df_estado_regiao = pd.read_csv('estado_regiao.csv')

# Exibe as primeiras linhas do DataFrame
print(df_estado_regiao.head())

  id_estado;sigla;estado;regiao;pais
0             1;AC;Acre;Norte;Brasil
1       2;AL;Alagoas;Nordeste;Brasil
2            3;AP;Amapá;Norte;Brasil
3         4;AM;Amazonas;Norte;Brasil
4         5;BA;Bahia;Nordeste;Brasil
